In [2]:
require 'statsample-glm'
require 'open-uri'

content = open('http://www.ats.ucla.edu/stat/data/binary.csv')
File.write('binary.csv', content.read)

df = Daru::DataFrame.from_csv "binary.csv"
df.vectors = Daru::Index.new(['admit', 'gpa', 'gre', 'rank'])
df.to_category 'rank'
df.first 5

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"https://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"https://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    w

#<Daru::DataFrame(5x4)>
       admit   gpa   gre  rank
     0     0   380  3.61     3
     1     1   660  3.67     3
     2     1   800     4     1
     3     1   640  3.19     4
     4     0   520  2.93     4

In [3]:
formula = 'admit ~ gpa + gre + rank'
glm = Statsample::GLM::Regression.new formula, df, :logistic, epsilon: 1e-2, constant: 1
glm.model.coefficients :hash

{:rank_1=>1.3402039132410228, :rank_4=>-0.21125974461860883, :rank_2=>0.6647609856203268, :gpa=>0.002264425767848877, :gre=>0.8040375445991668, :constant=>-5.33018295935774}

In [4]:

new_data = Daru::DataFrame.new({
  'gre' => [df['gre'].mean]*4,
  'gpa' => [df['gpa'].mean]*4,
  'rank' => [1,2,3,4]
  })



#<Daru::DataFrame(4x3)>
                   gpa        gre       rank
          0      587.7 3.38990000          1
          1      587.7 3.38990000          2
          2      587.7 3.38990000          3
          3      587.7 3.38990000          4

In [5]:
new_data.to_category 'rank'
glm.predict new_data

#<Daru::Vector(4)>
                   0 0.21861205009740323
                   1  0.5166016076489643
                   2 0.18466844661001103
                   3 0.35228458931274254